***Vorlesung 'Syntax natürlicher Sprachen'***

--- 
# Vorlesung 2: Einführung Parsing 

- **Parsing: algorithmisches Verfahren zur Verarbeitung formaler Grammatik**
- **Ziel: Finden einer Ableitung aus den Regeln der Grammatik zu einem Input-Satz** (Überprüfung auf Wohlgeformtheit)


- **formale Grammatik: Regeln zur Beschreibung syntaktischer Strukturen**
  - ermöglicht **Generierung** aller wohlgeformten Sätze der dadurch beschriebenen Sprache
  - kompakte Darstellung (durch wiederholte Regelanwendung und Rekursion kann eine unendliche Menge an Sätzen generiert werden)
  - **Konstituentenregeln** (CFG) oder **Dependenzregeln** (*Dependency Grammar*)!


- https://www.nltk.org/book/ch08.html#summary

In [1]:
import nltk

---
## 1. Generierung
- Ableitung von Sätzen ausgehend von Startsymbol `S` (vgl. top-down-Parser)

---
### Formale Grammatik: CFG

- **CFG  = Kontextfreie Grammatik**
- auch: **Phrasenstrukturgrammatik**
- Konstituentenregeln (syntagmatische Struktur)

> Given a set of syntactic categories, a context-free grammar uses a set of productions to say how a phrase of some category A can be analyzed into a sequence of smaller parts. (https://www.nltk.org/book/ch08.html#summary)


In [2]:
grammar = nltk.CFG.fromstring("""
## Syntaktische Regeln:
    S -> NP VP
    PP -> P NP
    NP -> Det N
    NP -> NP PP
    NP -> Pron
    VP -> V NP | VP PP
##Lexikalische Regeln:
    Pron -> 'I'      
    Det -> 'an' | 'my'
    N -> 'elephant' | 'pajamas'
    V -> 'shot'
    P -> 'in'
""")

In [3]:
#Generierung (depth = maximale Anzahl an Regelanwendungen):
from nltk.parse.generate import generate
for sentence in generate(grammar, depth=5):
    print(' '.join(sentence))

an elephant shot an elephant
an elephant shot an pajamas
an elephant shot my elephant
an elephant shot my pajamas
an elephant shot I
an pajamas shot an elephant
an pajamas shot an pajamas
an pajamas shot my elephant
an pajamas shot my pajamas
an pajamas shot I
my elephant shot an elephant
my elephant shot an pajamas
my elephant shot my elephant
my elephant shot my pajamas
my elephant shot I
my pajamas shot an elephant
my pajamas shot an pajamas
my pajamas shot my elephant
my pajamas shot my pajamas
my pajamas shot I
I shot an elephant
I shot an pajamas
I shot my elephant
I shot my pajamas
I shot I


---
## 2. CFG-Parsing: Ambiguität 


### Verwendung CFG-Parser (Beispiel: PP-Attachment-Ambiguität)

- verwendeter Parser: Chart-Parser

In [4]:
sent = 'I shot an elephant in my pajamas'.split()
print(sent)

['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']


In [5]:
parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print(unicodelines=True)

(S
  (NP (Pron I))
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
      S                                       
 ┌────┴──────────────┐                         
 │                   VP                       
 │         ┌─────────┴──────────┐              
 │         VP                   PP            
 │    ┌────┴───┐            ┌───┴───┐          
 NP   │        NP           │       NP        
 │    │    ┌───┴─────┐      │   ┌───┴─────┐    
Pron  V   Det        N      P  Det        N   
 │    │    │         │      │   │         │    
 I   shot  an     elephant  in  my     pajamas

(S
  (NP (Pron I))
  (VP
    (V shot)
    (NP
      (NP (Det an) (N elephant))
      (PP (P in) (NP (Det my) (N pajamas))))))
      S                                       
 ┌────┴──────────────┐                         
 │                   VP                       
 │    ┌──────────────┴──────┐                  
 │    │                     NP                


---
### Koordinationsambiguität

In [6]:
grammar = nltk.CFG.fromstring("""
## linke Seite erster Regel = Startsymbol:
    NP -> NP CON NP
    NP -> Adj NP
    NP -> N
    CON -> 'und'
    Adj -> 'alte'
    N -> 'Männer' | 'Frauen'
""")

sent = 'alte Männer und Frauen'.split()

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print(unicodelines=True)

(NP (NP (Adj alte) (NP (N Männer))) (CON und) (NP (N Frauen)))
           NP             
      ┌────┴─────┬────┐    
      NP         │    │   
 ┌────┴────┐     │    │    
 │         NP    │    NP  
 │         │     │    │    
Adj        N    CON   N   
 │         │     │    │    
alte     Männer und Frauen

(NP (Adj alte) (NP (NP (N Männer)) (CON und) (NP (N Frauen))))
             NP           
 ┌───────────┴───┐         
 │               NP       
 │     ┌─────────┼────┐    
 │     NP        │    NP  
 │     │         │    │    
Adj    N        CON   N   
 │     │         │    │    
alte Männer     und Frauen



---
## 3. CFG-Parsing: Rekursion und Backtracking


---

### Verarbeitungsproblem 1:  Linksrekursion (nicht mit RecursiveDescent-Parser möglich)

- linksrekursive Regeln: `NP -> NP PP`, `VP -> VP PP`

In [7]:
grammar = nltk.CFG.fromstring("""
## Syntaktische Regeln:
    S -> NP VP
    PP -> P NP
    NP -> Det N 
#rekursive Regel:
    NP -> NP PP
    NP -> Pron
    VP -> V NP
#rekursive Regel:    
    VP -> VP PP
##Lexikalische Regeln:
    Pron -> 'I'      
    Det -> 'an' | 'my'
    N -> 'elephant' | 'pajamas'
    V -> 'shot'
    P -> 'in'
""")

In [8]:
sent = 'I shot an elephant'.split()
print(sent)

['I', 'shot', 'an', 'elephant']


#### Recursive-Descent-Parser (einfacher top-down-Parser)
- https://www.nltk.org/book/ch08.html#recursive-descent-parsing

##### *Endlosschleife bei linksrekursiver Regel!*

In [9]:
#parser = nltk.RecursiveDescentParser(grammar,trace=2)
#for tree in parser.parse(sent):
#    print(tree)
#    tree.pretty_print(unicodelines=True)

```
Parsing 'I shot an elephant'
    [ * S ]
  E [ * NP VP ]
  E [ * Det N VP ]
  E [ * 'an' N VP ]
  E [ * 'my' N VP ]
  E [ * NP PP VP ]
  E [ * Det N PP VP ]
  E [ * 'an' N PP VP ]
  E [ * 'my' N PP VP ]
  E [ * NP PP PP VP ]
  E [ * Det N PP PP VP ]
  E [ * 'an' N PP PP VP ]
  E [ * 'my' N PP PP VP ]
  E [ * NP PP PP PP VP ]
  E [ * Det N PP PP PP VP ]
  E [ * 'an' N PP PP PP VP ]
  E [ * 'my' N PP PP PP VP ]
  E [ * NP PP PP PP PP VP ]
  E [ * Det N PP PP PP PP VP ]
  E [ * 'an' N PP PP PP PP VP ]
  E [ * 'my' N PP PP PP PP VP ]
  E [ * NP PP PP PP PP PP VP ]
  E [ * Det N PP PP PP PP PP VP ]
  E [ * 'an' N PP PP PP PP PP VP ]
  E [ * 'my' N PP PP PP PP PP VP ]
  E [ * NP PP PP PP PP PP PP VP ]
  E [ * Det N PP PP PP PP PP PP VP ]
  E [ * 'an' N PP PP PP PP PP PP VP ]
  E [ * 'my' N PP PP PP PP PP PP VP ]
  ...
```

---
### Verarbeitungsproblem 2: Temporale Ambiguität *(benötigt Backtracking)*

In [10]:
sent = 'the old man the boat'.split()

grammar = nltk.CFG.fromstring("""
## Syntaktische Regeln:
    S -> NP VP
    NP -> Det Adj N
    NP -> Det N
    VP -> V NP
##Lexikalische Regeln:
    Det -> 'the'
    Adj -> 'old'    
    N -> 'man' | 'boat' | 'old'
    V -> 'man'
""")

parser = nltk.ChartParser(grammar,trace=0)

for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print(unicodelines=True)

(S (NP (Det the) (N old)) (VP (V man) (NP (Det the) (N boat))))
             S              
     ┌───────┴───┐           
     │           VP         
     │       ┌───┴───┐       
     NP      │       NP     
 ┌───┴───┐   │   ┌───┴───┐   
Det      N   V  Det      N  
 │       │   │   │       │   
the     old man the     boat



#### ShiftReduceParser (einfacher bottom-up-Parser):  im NLTK ohne Backtracking implementiert

##### findet keinen Parse!

In [11]:
#NLTK-ShiftReduceParser (bottom-up-Parser): kein Backtracking! 
#bleibt bei Analyse NP NP ((the old man) (the boat)) stehen, findet keinen vollständigen Parse

parser = nltk.ShiftReduceParser(grammar,trace=2)

for tree in parser.parse(sent):
    print(tree)
    tree.pretty_print(unicodelines=True)

Parsing 'the old man the boat'
    [ * the old man the boat]
  S [ 'the' * old man the boat]
  R [ Det * old man the boat]
  S [ Det 'old' * man the boat]
  R [ Det Adj * man the boat]
  S [ Det Adj 'man' * the boat]
  R [ Det Adj N * the boat]
  R [ NP * the boat]
  S [ NP 'the' * boat]
  R [ NP Det * boat]
  S [ NP Det 'boat' * ]
  R [ NP Det N * ]
  R [ NP NP * ]


---


## 4. Dependency Parsing: Projektivität

--- 

### Formale Grammatik: Dependenzgrammatik

- formale Grammatik mit Regeln bzgl. Dependenzrelationen zwischen Wörtern (Subjekt, Objekt, Attribut usw.)
- abstrahiert von linearer Anordnung

> A dependency grammar uses productions to specify what the dependents are of a given lexical head. (https://www.nltk.org/book/ch08.html#summary)

- https://www.nltk.org/book/ch08.html#dependencies-and-dependency-grammar

In [12]:
grammar = nltk.DependencyGrammar.fromstring("""
    'beißen' -> 'Hunde' 
    'Hunde' -> 'bellen' 
    'bellen' -> 'die' 
    """)

### Beispiel Relativsatz

In [13]:
sent = 'Hunde    die bellen    beißen'.split()
print(sent)

['Hunde', 'die', 'bellen', 'beißen']


In [14]:
parser = nltk.ProjectiveDependencyParser(grammar)

for tree in parser.parse(sent):
    print(tree, "\n")
    tree.pretty_print(unicodelines=True)

(beißen (Hunde (bellen die))) 

beißen
  │    
Hunde 
  │    
bellen
  │    
 die  



---
### Verarbeitungsproblem: Nicht-projektive Strukturen

- auch: *long distance dependency*, diskontinuierliche Strukturen
    - vgl. https://en.wikipedia.org/wiki/Discontinuity_(linguistics)#Projectivity


- mit CFGs nicht direkt modellierbar (überkreuzende Kanten im Syntaxbaum)


- können mit Dependenzgrammatiken modelliert werden (abstrahiert von linearer Anordnung) 
- aber: **nur bestimmte Dependency Parsingalgorithmen erlauben die Verarbeitung solcher Strukturen**


#### Beispiel: ins Nachfeld extrahierter Relativsatz (Rechtsversetzung):

In [15]:
sent = 'Hunde beißen   die bellen '.split()
print(sent)

['Hunde', 'beißen', 'die', 'bellen']


#### ProjectiveDependencyParser: kein Parse!

In [16]:
#kein Parse mit ProjectiveDependencyParser!
parser = nltk.ProjectiveDependencyParser(grammar)

for tree in parser.parse(sent):
    print(tree, "\n")
    tree.pretty_print(unicodelines=True)

#### aber mit NonprojectiveDependencyParser:

In [17]:
#NonprojectiveDependencyParser: erkennt Struktur korrekt
#Code s. http://www.nltk.org/howto/dependency.html:

dp = nltk.NonprojectiveDependencyParser(grammar)
g, = dp.parse(sent)

print(g.root['word'])

for _, node in sorted(g.nodes.items()):
    if node['word'] is not None:
        print('{address} {word}: {d}'.format(d=node['deps'][''], **node))

print('\n', g.tree(), '\n')
g.tree().pretty_print(unicodelines=True)

beißen
1 Hunde: [4]
2 beißen: [1]
3 die: []
4 bellen: [3]

 (beißen (Hunde (bellen die))) 

beißen
  │    
Hunde 
  │    
bellen
  │    
 die  

